# Coordinate Reference System (CRS) Conversion
Using the library pyproj (http://pyproj4.github.io/pyproj/stable/) to transform the CRS you have to the CRS you want. The first example is for a single coordinate pair to be converted with an example to do 1,000's to 1,000,000's of transformations at once using Pandas.

### Libraries to use in the notebook

In [1]:
import pandas as pd
from pyproj import Proj, Transformer, transform

### Example well
This well is from  Colorado Oil and Gas Conservation Commision well shapefile (https://cogcc.state.co.us/documents/data/downloads/gis/WELLS_SHP.ZIP)<br>
The longitude and latitude data infomation is in the NAD 1983 Datum<br><br>
Searching spatial reference.org for the European Petroleum Survey Group number, we get 4269 (https://spatialreference.org/ref/epsg/4269/)

In [2]:
##Input well
API = '0500105000'
NAD83Lat = 39.741587
NAD83Long = -103.727484
InCRS = 'EPSG:4269' ## NAD 83 Latitude and Longitude (units = degrees) https://spatialreference.org/ref/epsg/4269/

A common transform for colorado is using UTM 13N for the DJ Basin, which works well for most of colorado.
<img src=https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Utm-zones-USA.svg/1024px-Utm-zones-USA.svg.png  width="500"> (source: https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system)
So, we will make that our output CRS.

In [3]:
##Project CRS
OutCRS = 'EPSG:26713' ## UTM zone 13N in datum NAD27 (units = m) https://spatialreference.org/ref/epsg/26713/

### Checking the CRS values we have selected with pyproj
Using the Proj tool, we can see if the EPSG numbers are providing us the proper projections

In [4]:
proj_in = Proj(InCRS)
proj_out = Proj(OutCRS)
print(proj_in)
print(proj_out)

Proj('+proj=longlat +datum=NAD83 +no_defs', preserve_units=True)
Proj('+proj=utm +zone=13 +datum=NAD27 +units=m +no_defs', preserve_units=True)


### Transforming the coordinates 
First define the transformer to be completed, then the coordinates to transform to desired CRS.

In [5]:
transformer = Transformer.from_crs(InCRS, OutCRS) ##transformer based on input and output CRS
UTM27_13N_East, UTM27_13N_North = transformer.transform(NAD83Lat, NAD83Long) ##Easting and Northing result from transformed Lat-Long
print('original:',API, NAD83Lat, NAD83Long) ##Original Data for comparison
print('transformed:',API, UTM27_13N_East,UTM27_13N_North) ##Transformed data

original: 0500105000 39.741587 -103.727484
transformed: 0500105000 609079.8939495534 4399643.509854254


Knowing how this works is great, but it will take way too long to convert single pairs at a time.

<br><br>
## Now lets setup to do this at project scale
<br><br>

### Loading the data to a dataframe
A file of lat-long data for 100 Colorado wells is in the Data Directory inside the directory for this notebook on github, make sure to have that downloaded.

In [6]:
path = 'Data//API-Lat-Long.txt'  ##Data directory in same directory as notebook
types_dict = {'API': str, 'NAD83Lat': float, 'NAD83Long': float}  ## dict with data formats for the file
df = pd.read_csv(path, sep='\t', names = ['API', 'NAD83Lat', 'NAD83Long'], dtype=types_dict) ##Tab separation input file with API, Lat, Long
df.head()  ##display the first 5 rows of the dataframe

,API,NAD83Lat,NAD83Long
0,0500105000,39.741587,-103.727484
1,0500105001,39.745257,-103.713113
2,0500105002,39.747303,-104.744385
3,0500105003,39.748516,-103.910839
4,0500105004,39.752736,-104.008242


### Data source

Again, this is from the Colorado Oil and Gas Conservation Commision well shapefile (https://cogcc.state.co.us/documents/data/downloads/gis/WELLS_SHP.ZIP)<br>
The longitude and latitude data infomation is in the NAD 1983 Datum.

### Other Common CRS
From https://spatialreference.org/<br>
WGS84 Lat Long: EPSG:4326 <br>
NAD27 Lat Long: EPSG:4267 <br>
NAD83 Lat Long: EPSG:4269 ##Current Lat Long data <br>
UTM13N NAD27: EPSG:26713 <br>

Many more can be found and I encourage you to test different ones for your project needs.

### Function to do the work
Here is a function that will do the work for us.<br>
The function needs specific inputs:<br>
        row = in a dataframe row<br>
        InCRS = input CRS<br>
        OutCRS = output CRS<br>
        Lat_E_Col = the column number or name for Latitude or Easting (remember python starts number with 0)<br>
        Long_N_Col = the column number or name for Longitude or Northing (remember python starts number with 0) <br>

In [7]:
def convertLatLong(row, InCRS, OutCRS, Lat_E_Col, Long_N_Col):
    transformer = Transformer.from_crs(InCRS, OutCRS)  ## transformation definition
    x,y = transformer.transform(row[Lat_E_Col], row[Long_N_Col]) ## x is output Lat or Easting, y is output Long or Northing
    return (x,y) ##return of values

#### That's it. Put in data and what you want and in two lines you get the conversion complete. <br><br>

### Example of using the function
Using the same types of inputs with the dataframe. .apply and zip allow the conversion to be done within the dataframe and get the new data entered.

In [9]:
InCRS = 'EPSG:4269'
OutCRS = 'EPSG:4267'
Lat_E_Col = 'NAD83Lat'  ## could also send the column reference number 1
Long_N_Col = 'NAD83Long'  ##could also send the column reference number 2


df['NAD27Lat'], df['NAD27Long'] = zip(*df.apply(convertLatLong, args=(InCRS, OutCRS, Lat_E_Col, Long_N_Col), axis=1))
df.head()

,API,NAD83Lat,NAD83Long,NAD27Lat,NAD27Long
0,0500105000,39.741587,-103.727484,39.741602,-103.726953
1,0500105001,39.745257,-103.713113,39.745272,-103.712583
2,0500105002,39.747303,-104.744385,39.747323,-104.743823
3,0500105003,39.748516,-103.910839,39.748532,-103.910302
4,0500105004,39.752736,-104.008242,39.752753,-104.007702


### Example for UTM outputs
Using a shortened input line with column numbers and rounding the output results to make the Easting and Northing to outputs that are more appropriate to significan digits

In [10]:
##shortening input to single line and using column references
## original NAD83 lat long to UTM zone 12N in the NAD27 datum
## Using the column number to call the correct lat-long for conversion
InCRS, OutCRS, LatCol, LongCol = 'EPSG:4369','EPSG:26713',1,2

## Getting the Easting and Northing from the function above for the dataframe
df['UTM27_13N_East_m'], df['UTM27_13N_North_m'] = zip(*df.apply(convertLatLong, args=(InCRS, OutCRS, LatCol, LongCol), axis=1))

## rounding UTM coordinates to two decimal places for display, and honestly do you need more than a 1 cm accuracy?
df['UTM27_13N_East_m'], df['UTM27_13N_North_m'] = df['UTM27_13N_East_m'].round(2), df['UTM27_13N_North_m'].round(2)
df.head()

,API,NAD83Lat,NAD83Long,NAD27Lat,NAD27Long,UTM27_13N_East_m,UTM27_13N_North_m
0,0500105000,39.741587,-103.727484,39.741602,-103.726953,609034.43,4399641.19
1,0500105001,39.745257,-103.713113,39.745272,-103.712583,610259.99,4400066.11
2,0500105002,39.747303,-104.744385,39.747323,-104.743823,521900.02,4399532.64
3,0500105003,39.748516,-103.910839,39.748532,-103.910302,593314.09,4400203.18
4,0500105004,39.752736,-104.008242,39.752753,-104.007702,584963.72,4400574.63


### Using those Imperial units
Just because I have create more than my share of projects that required UTM-m to be in feet, here is a quick conversion method.

In [11]:
df['UTM27_13N_East_ft'], df['UTM27_13N_North_ft'] = (df['UTM27_13N_East_m']*3.28084).round(2), (df['UTM27_13N_North_m']*3.28084).round(2) 
df.head()

,API,NAD83Lat,NAD83Long,NAD27Lat,NAD27Long,UTM27_13N_East_m,UTM27_13N_North_m,UTM27_13N_East_ft,UTM27_13N_North_ft
0,0500105000,39.741587,-103.727484,39.741602,-103.726953,609034.43,4399641.19,1998144.52,14434518.80
1,0500105001,39.745257,-103.713113,39.745272,-103.712583,610259.99,4400066.11,2002165.39,14435912.90
2,0500105002,39.747303,-104.744385,39.747323,-104.743823,521900.02,4399532.64,1712270.46,14434162.67
3,0500105003,39.748516,-103.910839,39.748532,-103.910302,593314.09,4400203.18,1946568.60,14436362.60
4,0500105004,39.752736,-104.008242,39.752753,-104.007702,584963.72,4400574.63,1919172.37,14437581.27


## Conclusion
This is a fairly simple way to get the CRS conversions you need quickly. If you know you current CRS and your project CRS, pyproj and pandas makes it fairly easy to do the conversion and stay organized.

It is also possible to do this work with geopandas (https://geopandas.org/). That could be added to the Utility Script GitHub in the future. With geopandas it stores the coordinates in a single column, so it isn't as necessary to keep the individual lat or long columns.

If you have any questions please let me know on GitHub (https://github.com/mlbelobraydi) or on linkedin (www.linkedin.com/in/matthew-belobraydic)